In [ ]:
# path data yang diproses
path = '/content/drive/MyDrive/S3/Pemrosesan Bahasa Alami Lanjut/PR_02_NLP/'

**Langkah 1:** Data loading, analisis eksplorasi data dan preprocessing. Master dataset dari
https://www.kaggle.com/datatattle/covid-19-nlp-text-classification

Dataset sudah di preprocess terlebih dahulu, pengolahan dalam file ipynb terpisah yang di submit bersamaan


In [ ]:
import torch
import pandas as pd # Utk memanipulasi dataset
from tqdm.notebook import tqdm # Utk menampilkan progress pemrosesan data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load data training dan testing (pilih data dengan 5 atau 3 kelas)
'''
# load train and test data (5 Class)
train = pd.read_csv(path+'Corona_NLP_train_preprocessed.csv')
test = pd.read_csv(path+'Corona_NLP_test_preprocessed.csv')
'''
# load train and test data (3 Class)
train = pd.read_csv(path+'Corona_NLP_train_preprocessed_3class.csv')
test = pd.read_csv(path+'Corona_NLP_test_preprocessed_3class.csv')

# Select Column
train = train[['CleanText','Sentiment']]
train.columns = ['text', 'label']
test = test[['CleanText','Sentiment']]
test.columns = ['text', 'label']

In [ ]:
# Menampilkan lima row pertama

print(train.head())
print(test.head())

                                                text  label
0    _mention_ _mention_ _mention_ _url_ _url_ _url_      1
1  advice talk neighbours family exchange phone n...      2
2  coronavirus australia woolworths give elderly ...      2
3  food stock one empty please nt panic enough od...      2
4  ready go supermarket _hashtag_ outbreak parano...      0
                                                text  label
0  trending new workers encounter empty supermark...      0
1  could nt find hand sanitizer fred meyer turned...      2
2                  find protect loved ones _hashtag_      2
3  _hashtag_ buying hits _hashtag_ city anxious s...      0
4  _hashtag_ _hashtag_ _hashtag_ _hashtag_ _hasht...      1


In [ ]:
# Label dict, memberikan label unik untuk setiap sentimen

label_dict = train.label.unique()
print(label_dict)

[1 2 0]


**Langkah 2:** Training/Validation SPLIT

Problem umum, data imbalance. Pada kasus ini misalnya sentimen netral hanya kurang separuhnya dari jumlah sentimen negatif atau positif.
Solusi, split tiap kelas (stratify). Agar memastikan tiap kelas terwakili
baik di data training maupun di validasi

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Membagi data, 9% nya utk validasi dan sisa 91% adalah data training
# Perintah 'stratify' memastikan semua kelas terwakili pada kedua kategori data

X_train, X_val, y_train, y_val = train_test_split(
    train.index.values, # secara unik mengidentifikasi setiap sampel
    train.label.values, # split berdasarkan index dan label
    test_size=0.09, # ditentukan 91% untuk training data
    random_state=17, # untuk bisa reproduksi hasil yang nanti didapat
    stratify=train.label.values # memastikan distribusi data tiap kelas
)

In [ ]:
# Menambahkan satu kolom yaitu data_type utk nanti di assign label classnya

train['data_type'] = ['not_set']*train.shape[0] # data_type, apakah data training atau validasi
train.head()
test['data_type'] = ['not_set']*test.shape[0]
test.head() # menampilkan data yang sudah memiliki kolom baru (data_type)

,text,label,data_type
0,trending new workers encounter empty supermark...,0,not_set
1,could nt find hand sanitizer fred meyer turned...,2,not_set
2,find protect loved ones _hashtag_,2,not_set
3,_hashtag_ buying hits _hashtag_ city anxious s...,0,not_set
4,_hashtag_ _hashtag_ _hashtag_ _hashtag_ _hasht...,1,not_set


In [ ]:
# Assign label data_type dalam kategori train dan validasi
train.loc[X_train, 'data_type'] = 'train'
train.loc[X_val, 'data_type'] = 'val'
test.loc[:,'data_type'] = 'test'

In [ ]:
# Menampilkan distribusi dari data train dan data validasi
train.groupby(['label', 'data_type']).count()

text
label data_type       
0     train      14012
      val         1386
1     train       7016
      val          694
2     train      16422
      val         1624

**Langkah 3:** Loading Tokenizer dan Encoding Data

In [ ]:
!pip install transformers #install transformers

     |████████████████████████████████| 1.8MB 8.6MB/s 
     |████████████████████████████████| 3.2MB 35.0MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5b9da08f7ce34d14cd35474badd3a168665c3bce2dee9049dbd87a81e480f32d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install pytorch-pretrained-bert # install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 8.0MB/s 
     |████████████████████████████████| 133kB 13.7MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 7.2MB 13.7MB/s 
ERROR: botocore 1.20.6 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
from transformers import BertTokenizer # Tokenizer, raw texts -> tokens (numerical data, yg mewakili kata tertentu)
from torch.utils.data import TensorDataset # setup dataset yang akan digunakan PyTorch

In [ ]:
tokenizer = BertTokenizer.from_pretrained( # akan digunakan pre-trained BERT
    'bert-base-uncased', # all lower case data, dataset juga sudah di proses semua dalam lower case
    do_lower_case=True # hanya memastikan bahwa semua data dikonversi ke lower case
)

In [ ]:
# Tiga proses yang dilakukan secara terpisah, train, validasi dan test

encoded_data_train = tokenizer.batch_encode_plus (
    train[train.data_type=='train'].text.values, # parameter pertama, kalimat (tweets) nya bagian yg training
    add_special_tokens=True, # BERT mengetahui kapan kalimat berakhir dan dimulai
    return_attention_mask=True, # dimana actual valuenya dan dimana valuenya 0, kalimat berakhir
    pad_to_max_length=True, # pad semua tweets ke panjang maksimal (max length) tertentu
    max_length=256, # memastikan semua masuk dalam fix length data yg dioleh (maksimal 1 tweet 160 char, 256 pasti memenuhi)
    return_tensors='pt' # bagaimana kita mau return tensornya, pt singkatan pytorch 
)

# Yang lain hampir sama
# Sekarang untuk yang validasi (val)
encoded_data_val = tokenizer.batch_encode_plus (
    train[train.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Test data
encoded_data_test = tokenizer.batch_encode_plus (
    test[test.data_type=='test'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Split (encoded) data set menjadi

input_ids_train = encoded_data_train['input_ids'] # input_ids, represent tiap kata sebagai number
attention_masks_train = encoded_data_train['attention_mask'] # pytorch tensor
labels_train = torch.tensor(train[train.data_type=='train'].label.values) # membuat tensor dari original data

# Kedua dibawah prosesnya sama, hanya beda di Val dan Test

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train[train.data_type=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test[test.data_type=='test'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Dari langkah sblmnya, skrg create tiga dataset

dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train, labels_train) # cara standar (default) menggunakan dataset pada pytorch library
dataset_val = TensorDataset(input_ids_val,
                              attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test,
                              attention_masks_test, labels_test)

In [ ]:
# Mengecek jumlah dataset, training
len(dataset_train)

37450

In [ ]:
# Mengecek jumlah dataset, validasi
len(dataset_val)

3704

**Langkah 4:** Setting up BERT Pretrained Model
Pada prinsipnya, ini langkah fine tuning BERT dimana BERT yg sudah di pretrained di corpus yang amat besar, kita menambahkan layer diatasnya berukuran 3 (sesuai jumlah class) sesuai task yang ingin kita lakukan (tweet sentiment)


In [ ]:
# Memperlakukan setiap tweet dalam sekuens (unik) nya sendiri
# Tiap sekuens akan diklasifikasikan ke dalam salah satu class

from transformers import BertForSequenceClassification

In [ ]:
# Mendefinisikan ulang arsitektur
# num_labels menentukan berapa banyak output label 
model = BertForSequenceClassification.from_pretrained( # memanfaatkan pretrained BERT yang cukup lengkap
    'bert-base-uncased',  # base-bert dipilih, cukup utk belajar, lebih cepat utk komputasi
    num_labels = len(label_dict), # converted words -> number utk setiap label, berapa banyak final layer yang BERT punya utk mengklasifikasi
    output_attentions=False, # tidak memasukkan faktor attention utk kasus ini (tweet sentiment)
    output_hidden_states=False # state sebelum prediksi juga di ignore
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**Langkah 5:** Mambangun Data Loaders

In [ ]:
# Untuk sampling dataset per batch
# RandomSampler untuk training
# SequentialSampler untuk validasi

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
# Dataset dimasukkan ke Data Loader
# Ada tiga data loader, masing-masing train, val dan test

batch_size = 32 # Platfrm Colab menyediakan GPU & Memory, sehingga bisa menngunakan batch size 32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train), # mencegah model dari learning perbedaan sequence-based
    batch_size=batch_size
)

# Dua data loader prinsipnya sama
# Sehingga nanti dataset sudah siap di data loader utk tahapan selanjutnya

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

dataloader_test = DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=1
)

**Langkah 6:** Setting up Optimizer dan Scheduler

Mendefinisikan optimizer yang akan digunakan, optimasi bobot untuk output
Mendefinisikan scheduler, menyesuaikan learning rate selama training berjalan

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup # scheduler mengkontrol bagaimana learning rate berubah dengan berjalannya waktu


In [ ]:
optimizer = AdamW( # cara untuk utk optimizing weight (stochastic optimization approach)
    model.parameters(),
    lr=5e-5, #2e-5 sampai 5e-5 (range learning-rate yang direkomendasikan di paper)
    eps=1e-8 # default
)

In [ ]:
epochs = 15 # menyesuaikan dengan batasan Google Colab, 15 epochs ini dijalankan selama 7 jam

scheduler = get_linear_schedule_with_warmup( # set scheduler
    optimizer, # AdamW
    num_warmup_steps=0, # default
    num_training_steps=len(dataloader_train)*epochs # Jumlah iterasi, atau jumlah training steps, berapa kali learning rate berubah
)

**Langkah 7:** Mendefinisikan Performance Metric
Menggambarkan seberapa bagusnya performa model dari training

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
# Alasan digunakan f1 score karena ada imbalance data
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten() # single array
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted'), f1_score(labels_flat, preds_flat, average='macro') 
  # weighted average, weight tiap class berdasar berapa sampel yang ada

In [ ]:
def accuracy_per_class(preds, labels): # Utk print accuracy per class (yang bernilai true)
  label_dict_inverse = {v: k for k, v in label_dict.items()} # inverse label ke nama class nya utk identifikasi

  # prinsipnya sama spt langkah sblmnya
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat): # iterate semua label yg unik
    y_preds = preds_flat[labels_flat==label] # numpy indexing
    y_true = labels_flat[labels_flat==label] # konversi number (label) ke dalam class
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

**Langkah 8:** Membuat Training Loop
Training loop untuk fine tuning model BERT

In [ ]:
import random
seed_val = 17 # menjamin konsistensi seed
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# Memastikan GPU dipakai jika tersedia di device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
# function evaluate, proses step by stepnya amat mirip dengan training function (sblmnya)
def evaluate(dataloader_val):  

  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []

  for batch in tqdm(dataloader_val):

    batch = tuple(b.to(device) for b in batch)

    inputs = {'input_ids':      batch[0],
              'attention_mask': batch[1],
              'labels' :        batch[2],
              }

    with torch.no_grad():
      outputs = model (**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)

  loss_val_avg = loss_val_total/len(dataloader_val)

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)): # 15 epochs, tidak termasuk 16

  model.train() # Model dalam training mode (pytorch)
  
  # Set training loss ke 0, tiap epoch akan mendapatkan avg training loss
  # Dimulai dari 0 awalnya, lalu mulai menambahkan setiap loss pada batch pada variabel
  loss_train_total = 0

  # Untuk melihat berapa batch yang sudah di training, dan berapa sisanya
  # Untuk memastikan apakah progressnya sedang berjalan atau hang
  progress_bar = tqdm(dataloader_train,
                      desc='Epoch {:1d}'.format(epoch),
                      leave=False,
                      disable=False)
  
  # Utk setiap epoch kita akan menggunakan batches
  for batch in progress_bar:

      # Set gradient ke 0
      model.zero_grad()

      # Memastikan setiap tuple di device yang benar
      # Penting karena menggunakan GPU (cuda)
      batch = tuple(b.to(device) for b in batch)

      # Input adalah apa yang dimasukkan ke model
      inputs = {
          'input_ids'       : batch[0], # Item pertama di Tuple
          'attention_mask'  : batch[1],
          'labels'          : batch[2]
      }

      outputs = model(**inputs) # unpack dictionary ke input

      # Apa yang dikeluarkan oleh BERT model adalah loss dan logit (hidden layer units)

      loss = outputs[0] # loss
      loss_train_total += loss.item() # Total loss (train)
      loss.backward() # Back propagation

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # kontrol gradient

      optimizer.step()
      scheduler.step()

      # untuk tampilan training_loss
      # loss item/len(batch)
      progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
  
  # Save modelnya setelah epoch
  torch.save(model.state_dict(), path+'BERT_ft_epoch{epoch}.model') # simpan dlm format .model

tqdm.write('\nEpoch {epoch}')

# Menghitung performansi data training
loss_train_avg = loss_train_total/len(dataloader_train)
tqdm.write(f'Training loss: {loss_train_avg}') # Average training loss per epoch
train_loss, predictions, true_vals = evaluate(dataloader_train)
train_f1weight, train_f1macro = f1_score_func(predictions, true_vals)

# tampilan utk progress bar dan reporting
tqdm.write(f'F1 Score (weighted): {train_f1weight}')
tqdm.write(f'F1 Score (macro): {train_f1macro}')

# Menghitung performansi data validasi
val_loss, predictions, true_vals = evaluate(dataloader_val)
val_f1weight, val_f1macro = f1_score_func(predictions, true_vals)

# tampilan utk progress bar dan reporting
tqdm.write(f'Validation loss: {val_loss}')
tqdm.write(f'F1 Score (weighted): {val_f1weight}')
tqdm.write(f'F1 Score (macro): {val_f1macro}')



Epoch {epoch}
Training loss: 0.0029966346247675534



F1 Score (weighted): 0.9998664987128224
F1 Score (macro): 0.9998406169228634



Validation loss: 1.1513238227386142
F1 Score (weighted): 0.880140307289881
F1 Score (macro): 0.8713169450190691


**Langkah 9:** Loading dan mengevaluasi model (proses testing)

In [ ]:
# Load saved model
model.load_state_dict(torch.load(path+'BERT_ft_epoch{epoch}.model'))
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# Performansi sistem untuk data testing
test_loss , predictions, true_vals = evaluate(dataloader_test)
test_f1weight, test_f1macro = f1_score_func(predictions, true_vals)

# Tampilan utk progress bar dan reporting
tqdm.write(f'Test loss: {test_loss}')
tqdm.write(f'F1 Score (weighted): {test_f1weight}')
tqdm.write(f'F1 Score (macro): {test_f1macro}')


Test loss: 1.4443254175396143
F1 Score (weighted): 0.845046246580766
F1 Score (macro): 0.8319143960841013
